In [ ]:
import pymongo
import requests
from bs4 import BeautifulSoup as bs
import GetOldTweets3 as got
import time

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
import re, string
from nltk.tokenize import word_tokenize
from classifier import classifier

In [ ]:
# used to normalize raw tweets
def clean_data(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        # removing unwanted symbols and patterns from tokens using regular expressions
        token = re.sub("http[s]?://+(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+","", token)
        token = re.sub(".(.co/o).","", token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = re.sub("\w*\d\w*","",token)
        token = re.sub('[‘’“”…#–/]', '', token)
#         token = re.sub("http[s]?","",token)
        
        # assigning new pos tags for WordNetLemmatizer() function
        if tag.startswith("NN"):
            pos = "n"
        elif tag.startswith("VB"):
            pos = "v"
        else:
            pos = "a"
            
        # lemmatizing tokens (running=run)
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        # dropping puncuation and stop words
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_tweets(search_term, city, year):
    months = ["8"]
    all_tweets = []
    for month in months:
        try:
            # setting parameters
            tweetCriteria = got.manager.TweetCriteria() \
                .setQuerySearch(search_term) \
                .setSince(f"{year}-{month}-01") \
                .setUntil(f"{year}-{month}-28") \
                .setEmoji("unicode") \
                .setWithin("100mi") \
                .setNear(city) \
                .setMaxTweets(20)
#                 .setTopTweets(True) \

            # scraping tweets
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)

            # adding all tweets to list
            if len(tweets) != 0:
                all_tweets.append(tweets)

            # delay between calls
            time.sleep(3)
        except:
            print("Not Found")
    if len(all_tweets) != 0:
        tweets = [tweet for tweets in all_tweets for tweet in tweets]
        print(f"Tweets collected for {search_term}: {len(tweets)}")
    else:
        print(f"No tweets found for {search_term}")
        
    return tweets

### Connecting to MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.restaurant_db

### Scraping Tweets
* Placeholder 0:100

In [7]:
year = "2020"
record_count = 0

search_term = "Cheesecake Factory"
city = restaurant["location"]["city"]
alias = restaurant["alias"]

results = get_tweets(search_term, city, year)

if len(results) != 0:
    tweets = []
    for result in results:
        tweet = {}
        tokens = clean_data(word_tokenize(result.text))
        classifier_result = classifier.classify(dict([token, True] for token in tokens))
        tweet["date"] = result.date
        tweet["tweet"] = result.text
        tweet["tokens"] = tokens
        tweet["sentiment"] = classifier_result
        tweet["favorites"] = result.favorites
        tweet["replies"] = result.replies
        tweet["retweets"] = result.retweets
        tweet["hashtags"] = result.hashtags
        tweet["username"] = result.username
        tweet["geo"] = result.geo
        tweet["link"] = result.permalink
        tweets.append(tweet)


record_count+=1
print(f"Record Count: {record_count}")

An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=Cheesecake%20Factory%20near%3A%22Aguilar%22%20within%3A100mi%20since%3A2020-8-01%20until%3A2020-8-28&src=typd
Not Found
No tweets found for Cheesecake Factory


UnboundLocalError: local variable 'tweets' referenced before assignment

In [114]:
count = 0
for restaurant in db.yelp.find()[0:100]:
    count+=1
    print(f"{count} {restaurant['restaurant_name']}, {restaurant['location']['city']}")

1 Amato Bros Standard Truck Stop, Aguilar
2 Arby's, Alamosa
3 Burger King, Alamosa
4 Calvillo's Mexican Restaurant, Alamosa
5 Campus Cafe, Alamosa
6 Inn of the Rio Grande, Alamosa
7 Domino's Pizza, Alamosa
8 East West Grill, Alamosa
9 El Charro Cafe, Alamosa
10 Hideaway Steakhouse, Alamosa
11 Hunan Chinese Restaurant, Alamosa
12 KFC, Alamosa
13 Lamplighter Restaurant, Alamosa
14 Long John Silver's, Alamosa
15 McDonald's, Alamosa
16 Monterrey Cafe, Alamosa
17 Mrs Rivera's Kitchen, Alamosa
18 Nino's Del Sol, Alamosa
19 Pizza Den & Pasta, Alamosa
20 Sonic Drive-In, Alamosa
21 Subway, Alamosa
22 Subway, Alamosa
23 Valley Food Co-Op, Alamosa
24 Wendy's, Alamosa
25 The Fawn Brook Inn, Allenspark
26 Meadow Mountain Cafe, Allenspark
27 Almont Resort & Restaurant, Almont
28 Nugget Cafe, Almont
29 Dos Hermanas, Antonito
30 Dutch Mill Restaurant, Antonito
31 Asana Studio Yoga, Arvada
32 Alamos Verdes Restaurante, Arvada
33 Fanelli's Amici's, Wheat Ridge
34 Anthony's Pizza and Pasta, Arvada
35 App